In [1]:
import cv2
import numpy as np
import math

def GetEyes(frame):
    # Umwandeln in Graustufenbild
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Haar-Cascade-Datei für Augen laden
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    # Erkennen der Augen
    eyes = eye_cascade.detectMultiScale(grayFrame, scaleFactor=1.1, minNeighbors=4)
    print("Augen: ",eyes)
    return eyes

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngleAndSpacing(cords):
    print("Cords der Augen: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print("delta X Augen:", deltaX, "delta Y Augen: ", deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan))
    spacing = math.sqrt((deltaX ** 2 + deltaY ** 2))
    print("Abstand der Augen: ", spacing)
    return alpha, spacing

#Rotieren und Skalieren des Bildes mit berechnetem Winkel 
def RotateAndScale(image, angle, spacing):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

    #Bild Sklaieren auf den gleichen Augenabstand des linken Bildes
    scale_percent = 100 - (1-spacing/60)*100 #scale des Bildes in % CutFace hat einen Abstand von 60
    print("Skalierung in Prozent", scale_percent)
    width = int(image.shape[1] * scale_percent / 100) #Berechung der Breite
    height = int(image.shape[0] * scale_percent / 100) #Berechnung der Höhe
    dim = (width, height) #Tupel aus neuer Skalierung
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) #Resize Methode
    cv2.imwrite("Result.png", result)
    return result

#Überlappen der Beiden Bilder, damit die Augen aufeinander liegen
def Overlap(image1, image2):
    y = 480
    x = 640

    print("X Finale Image: ", x, " Y Finale Image: ", y)
    #Neues np Array anlegen, dass die größte Breite und die Größte Höhe hat
    result = np.zeros((y, x, 3), dtype=np.uint8)

    #Offset mit dem das GesichtFrau Bild verschoben werden soll
    xOffset = 100
    yOffset = 100

    #Durchiterrieren aller Pixel um das Finale Bild generieren zu lassen result ist ein Bild mit RGB Kanal
    for i in range(x):
        for j in range(y):
            #try fängt indizierungen auserhalb des erlaubten Bereichs ab
            try:
                if image2[j][i][3] == 0 and j-yOffset < 0 and i-xOffset < 0:
                    result[j][i]  = [image1[j-yOffset][i-xOffset][0],image1[j-yOffset][i-xOffset][1],image1[j-yOffset][i-xOffset][2]]

                else:
                    result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
            except:
                result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
    return result
  
vid = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()

    #Augen Position erfassen
    eyes = GetEyes(frame)

    for (x, y, w, h) in eyes:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    #Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
    eyesPos = list()
    for i in range(len(eyes)):
        eyesPos.append((eyes[i][0],eyes[i][1]))

    #Berechnen des Winkels und Abstand der Augen
    if len(eyesPos) == 2:
        angle, spacing = CalcAngleAndSpacing(eyesPos)
        print("Winkel", angle)
        print("Abstand der Augen", spacing)
        face = cv2.imread('CutFace.png', cv2.IMREAD_UNCHANGED)
        #Gesicht Rotieren und auf den Augenabstad skalieren
        rotatet = RotateAndScale(face, angle, spacing)
        #finalFrame = Overlap(frame, rotatet)
    
    #Anzeigen des Frames
    #print("Frame Shape: ", frame.shape)
    cv2.imshow('Pic', rotatet)
    cv2.imshow('Video', frame)
    #cv2.imshow('Video', finalFrame)
      
    #Beenden mit der Taste q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

Augen:  [[321 156  53  53]]


NameError: name 'rotatet' is not defined

In [ ]:
deltaXMin = 0
finalEyes = list()
for i in range(2,len(eyesPos)+1):
    print(i)
    delta = eyesPos[i-1][0] - eyesPos[i][0]
    if deltaXMin == 0 or delta < deltaXMin:
        deltaXMin = delta
        finalEyes = []
        finalEyes.append(eyesPos[i-1])
        finalEyes.append(eyesPos[i])

# Funktioniert aber unperformant

In [2]:
import cv2
import numpy as np
import math

def GetEyes(frame):
    # Umwandeln in Graustufenbild
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Haar-Cascade-Datei für Augen laden
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    # Erkennen der Augen
    eyes = eye_cascade.detectMultiScale(grayFrame, scaleFactor=1.1, minNeighbors=4)
    #print("Augen: ",eyes)
    return eyes

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngleAndSpacing(cords):
    print("Cords der Augen: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print("delta X Augen:", deltaX, "delta Y Augen: ", deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan)) * (-1)
    spacing = math.sqrt((deltaX ** 2 + deltaY ** 2))
    print("Abstand der Augen: ", spacing)
    return alpha, spacing

#Rotieren und Skalieren des Bildes mit berechnetem Winkel 
def RotateAndScale(image, angle, spacing):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

    #Bild Sklaieren auf den gleichen Augenabstand des linken Bildes
    scale_percent = 100 - (1-spacing/60)*100 #scale des Bildes in % CutFace hat einen Abstand von 60
    print("Skalierung in Prozent", scale_percent)
    width = int(image.shape[1] * scale_percent / 100) #Berechung der Breite
    height = int(image.shape[0] * scale_percent / 100) #Berechnung der Höhe
    dim = (width, height) #Tupel aus neuer Skalierung
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) #Resize Methode
    cv2.imwrite("Result.png", result)
    return result

#Überlappen der Beiden Bilder, damit die Augen aufeinander liegen
def Overlap(image1, image2):
    y = 480
    x = 720

    print("X Finale Image: ", x, " Y Finale Image: ", y)
    #Neues np Array anlegen, dass die größte Breite und die Größte Höhe hat
    result = np.zeros((y, x, 3), dtype=np.uint8)

    #Offset mit dem das GesichtFrau Bild verschoben werden soll
    xOffset = 900
    yOffset = 500

    #Durchiterrieren aller Pixel um das Finale Bild generieren zu lassen result ist ein Bild mit RGB Kanal
    for i in range(x):
        for j in range(y):
            #try fängt indizierungen auserhalb des erlaubten Bereichs ab
            try:
                if image2[j][i][3] == 0 and j-yOffset < 0 and i-xOffset < 0:
                    result[j][i]  = [image1[j-yOffset][i-xOffset][0],image1[j-yOffset][i-xOffset][1],image1[j-yOffset][i-xOffset][2]]

                else:
                    result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
            except:
                result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
    return result

def NewSized(image):
    contours, hierarchy = cv2.findContours(image[:,:,3], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    right = np.max([np.max(contour[:,:,0]) for contour in contours])
    bottom = np.max([np.max(contour[:,:,1]) for contour in contours])
    left = np.min([np.min(contour[:,:,0]) for contour in contours])
    top = np.min([np.min(contour[:,:,1]) for contour in contours])

    croppedImage = image[top:bottom, left:right]
    return croppedImage



# Video öffnen
cap = cv2.VideoCapture('TestVid.mp4')

# Solange das Video läuft
while(cap.isOpened()):

    # Frame für Frame auslesen
    ret, frame = cap.read()

    # Wenn das Video beendet ist, Schleife abbrechen
    if not ret:
        break

    #Augen Position erfassen
    eyes = GetEyes(frame)

    #Falsch erkannte Objekte herausfiltern die nicht in den möglichen Pixelbereich liegen können
    eyesFiltert = list()
    for i in range(len(eyes)):
        if eyes[i][3] > 50 and eyes[i][1] < 250 and eyes[i][1] > 150:
            eyesFiltert.append(eyes[i])
    #print(eyesFiltert)

    for (x, y, w, h) in eyesFiltert:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    #Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
    eyesPos = list()
    for i in range(len(eyesFiltert)):
            eyesPos.append((eyesFiltert[i][0],eyesFiltert[i][1]))

    #Berechnen des Winkels und Abstand der Augen
    if len(eyesPos) == 2:
        angle, spacing = CalcAngleAndSpacing(eyesPos)
        print("Winkel", angle)
        print("Abstand der Augen", spacing)
        face = cv2.imread('CutFace.png', cv2.IMREAD_UNCHANGED)
        #Gesicht Rotieren und auf den Augenabstad skalieren
        rotatet = RotateAndScale(face, angle, spacing)
        newSized = NewSized(rotatet)
        #frame = Overlap(frame, rotatet)
    
    #Anzeigen des Frames
    #print("Frame Shape: ", frame.shape)q
    cv2.imshow('Pic', newSized)
    cv2.imshow('Video', frame)

    # Auf Tastendruck warten, um das Fenster zu schließen
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

# Ressourcen freigeben
cap.release()
cv2.destroyAllWindows()


Cords der Augen:  [(214, 192), (313, 199)]
delta X Augen: -99 delta Y Augen:  -7
Abstand der Augen:  99.24716620639605
Winkel -4.057981600901857
Abstand der Augen 99.24716620639605
Skalierung in Prozent 165.41194367732675


QObject::moveToThread: Current thread (0x1a94f00) is not the object's thread (0x24b4ee0).
Cannot move to target thread (0x1a94f00)

Qt: Session management error: None of the authentication protocols specified are supported
QObject::moveToThread: Current thread (0x1a94f00) is not the object's thread (0x24b4ee0).
Cannot move to target thread (0x1a94f00)

QObject::moveToThread: Current thread (0x1a94f00) is not the object's thread (0x24b4ee0).
Cannot move to target thread (0x1a94f00)

QObject::moveToThread: Current thread (0x1a94f00) is not the object's thread (0x24b4ee0).
Cannot move to target thread (0x1a94f00)

QObject::moveToThread: Current thread (0x1a94f00) is not the object's thread (0x24b4ee0).
Cannot move to target thread (0x1a94f00)

QObject::moveToThread: Current thread (0x1a94f00) is not the object's thread (0x24b4ee0).
Cannot move to target thread (0x1a94f00)

QObject::moveToThread: Current thread (0x1a94f00) is not the object's thread (0x24b4ee0).
Cannot move to target threa

Cords der Augen:  [(214, 193), (312, 199)]
delta X Augen: -98 delta Y Augen:  -6
Abstand der Augen:  98.18350166906862
Winkel -3.5122945047903285
Abstand der Augen 98.18350166906862
Skalierung in Prozent 163.6391694484477
Cords der Augen:  [(214, 194), (313, 199)]
delta X Augen: -99 delta Y Augen:  -5
Abstand der Augen:  99.12618221237011
Winkel -2.896189151487238
Abstand der Augen 99.12618221237011
Skalierung in Prozent 165.2103036872835
Cords der Augen:  [(213, 193), (312, 198)]
delta X Augen: -99 delta Y Augen:  -5
Abstand der Augen:  99.12618221237011
Winkel -2.896189151487238
Abstand der Augen 99.12618221237011
Skalierung in Prozent 165.2103036872835
Cords der Augen:  [(213, 195), (312, 199)]
delta X Augen: -99 delta Y Augen:  -4
Abstand der Augen:  99.08077512817509
Winkel -2.316241538621982
Abstand der Augen 99.08077512817509
Skalierung in Prozent 165.13462521362516
Cords der Augen:  [(214, 195), (311, 199)]
delta X Augen: -97 delta Y Augen:  -4
Abstand der Augen:  97.0824391947

# Png Gesicht an den Rändern zuschneiden

In [4]:
import numpy as np
import cv2
image = cv2.imread('CutFace.png', cv2.IMREAD_UNCHANGED)

contours, hierarchy = cv2.findContours(image[:,:,3], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

right = np.max([np.max(contour[:,:,0]) for contour in contours])
bottom = np.max([np.max(contour[:,:,1]) for contour in contours])
left = np.min([np.min(contour[:,:,0]) for contour in contours])
top = np.min([np.min(contour[:,:,1]) for contour in contours])


croppedImage = image[top:bottom, left:right]

# Anzeigen des Ergebnisbildes
cv2.imshow("Cropped Image", croppedImage)
cv2.waitKey(0)
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x255e760) is not the object's thread (0x2e74d40).
Cannot move to target thread (0x255e760)

QObject::moveToThread: Current thread (0x255e760) is not the object's thread (0x2e74d40).
Cannot move to target thread (0x255e760)

QObject::moveToThread: Current thread (0x255e760) is not the object's thread (0x2e74d40).
Cannot move to target thread (0x255e760)

QObject::moveToThread: Current thread (0x255e760) is not the object's thread (0x2e74d40).
Cannot move to target thread (0x255e760)

QObject::moveToThread: Current thread (0x255e760) is not the object's thread (0x2e74d40).
Cannot move to target thread (0x255e760)

QObject::moveToThread: Current thread (0x255e760) is not the object's thread (0x2e74d40).
Cannot move to target thread (0x255e760)

QObject::moveToThread: Current thread (0x255e760) is not the object's thread (0x2e74d40).
Cannot move to target thread (0x255e760)

QObject::moveToThread: Current thread (0x255e760) is not the object's thread

In [50]:
import numpy as np
import cv2 as cv
im = cv.imread('CutFace.png', cv2.IMREAD_UNCHANGED)
print(im.shape)
#print(im[:,:,3].shape)

assert im is not None, "file could not be read, check with os.path.exists()"

contours, hierarchy = cv.findContours(im[:,:,3], cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

right = np.max([np.max(contour[:,:,0]) for contour in contours])
bottom = np.max([np.max(contour[:,:,1]) for contour in contours])
left = np.min([np.min(contour[:,:,0]) for contour in contours])
top = np.min([np.min(contour[:,:,1]) for contour in contours])

print("Max x: ", max_x)
print("Max y: ", max_y)
print("Min x: ", min_x)
print("Min y: ", min_y)

(408, 612, 4)
Max x:  395
Max y:  247
Min x:  227
Min y:  16
